## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [3]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [8]:
# 讀取Wine資料集
datas=datasets.load_diabetes()

In [9]:
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(datas.data, datas.target, test_size=0.25, random_state=42)

# 建立模型
clf = GradientBoostingRegressor(random_state=7)

In [10]:
y_train

array([ 166.,  189.,  173.,  220.,  206.,   97.,   60.,   61.,  242.,
        121.,  128.,  104.,  265.,  132.,  283.,  174.,  129.,  257.,
        137.,   63.,   93.,  232.,  208.,  261.,  179.,  258.,  262.,
         51.,  237.,   71.,  139.,  268.,   69.,  317.,  249.,  154.,
        192.,  116.,   81.,  122.,  259.,  191.,  292.,   55.,  107.,
        210.,   91.,  253.,   85.,  252.,   59.,   78.,  200.,   78.,
        245.,  175.,   42.,  127.,   53.,   94.,  104.,  199.,  265.,
        281.,  248.,  257.,  215.,  303.,  170.,   59.,  277.,  209.,
        138.,  198.,  124.,   96.,  288.,  225.,  265.,  101.,   55.,
        198.,   51.,  252.,   64.,  220.,  131.,  212.,  142.,  103.,
        155.,  121.,   86.,  111.,   65.,  131.,   51.,  128.,  141.,
         48.,  109.,  178.,   88.,   84.,  216.,  150.,   60.,   96.,
        190.,   74.,  279.,  182.,  160.,  245.,  276.,  174.,  180.,
        150.,  196.,  138.,   97.,  246.,  321.,  308.,  109.,   69.,
        182.,  258.,

In [11]:
# 先看看使用預設參數得到的結果，約為 8.379 的 MSE
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(metrics.mean_squared_error(y_test, y_pred))

3195.48463653


In [12]:
# 設定要訓練的超參數組合
n_estimators = [100, 200, 300,400,500,66]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

C:\Users\danlee\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  20 out of  27 | elapsed:    0.9s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    1.1s finished
C:\Users\danlee\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [13]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -3351.749150 using {'max_depth': 1, 'n_estimators': 100}


In [15]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

In [16]:
# 調整參數後約可降至 ? 的 MSE
print(metrics.mean_squared_error(y_test, y_pred))

2799.57746026
